In [26]:
import os, time, copy
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import normalize
from scipy.optimize import brentq
from scipy.stats import binom
from tqdm import tqdm

In [46]:
# Get data 2006-2014 from the following link: https://darchive.mblwhoilibrary.org/handle/1912/7341
# Unzip and merge the datasets in the following directory
calib_data = np.load('./calib-outputs.npz')
test_data = np.load('./test-outputs.npz')
classes = np.load('./classes.npy')
calib_preds, calib_labels = calib_data['preds'], calib_data['labels']
test_preds, test_labels = test_data['preds'], test_data['labels']

combined_preds = np.concatenate([calib_preds, test_preds])
combined_labels = np.concatenate([calib_labels, test_labels]).astype(int)
uq_labels, counts_labels = np.unique(combined_labels, return_counts = True)
num_labels = uq_labels.shape[0]

# Randomly sample a fraction of each label!
fractions_to_sample = np.random.random(size=(num_labels))

calib_idxs = np.concatenate([ np.random.choice(np.where(combined_labels == uq_labels[j])[0], size=(int(fractions_to_sample[j]*counts_labels[j]),), replace=False) for j in range(num_labels) ])
test_idxs = np.arange(combined_labels.shape[0]); test_idxs = test_idxs[np.logical_not(np.isin(test_idxs, calib_idxs))]

calib_preds = combined_preds[calib_idxs]
calib_labels = combined_labels[calib_idxs]
test_preds = combined_preds[test_idxs]
test_labels = combined_labels[test_idxs]

In [41]:
# Convert to binary
plankton_classes = np.array(np.where(np.isin(classes,['mix','mix_elongated','detritus','bad', 'bead', 'bubble', 'other_interaction', 'pollen', 'spore'],invert=True))[0])
calib_preds = np.isin(calib_preds, plankton_classes)
calib_labels = np.isin(calib_labels, plankton_classes)
test_preds = np.isin(test_preds, plankton_classes)
test_labels = np.isin(test_labels, plankton_classes)

In [42]:
n_y1 = calib_labels.sum()
n_y0 = calib_labels.shape[0]-calib_labels.sum()
N = test_preds.shape[0]

n_f0y0 = ((calib_labels == 0) & (calib_preds == 0)).astype(int).sum()
n_f1y1 = ((calib_labels == 1) & (calib_preds == 1)).astype(int).sum()
N_f1 = test_preds.sum()
N_f0 = N-N_f1

print(f"calib acc among non-plankton: {n_f0y0/n_y0:.3f} | calib acc among plankton: {n_f1y1/n_y1:.3f} | predicted fraction plankton: {N_f1/N:.3f}")

calib acc among non-plankton: 0.992 | calib acc among plankton: 0.665 | predicted fraction plankton: 0.114


In [43]:
test_n_y1 = test_labels.sum()
test_n_y0 = test_labels.shape[0]-test_labels.sum()

test_n_f0y0 = ((test_labels == 0) & (test_preds == 0)).astype(int).sum()
test_n_f1y1 = ((test_labels == 1) & (test_preds == 1)).astype(int).sum()

print(f"test acc among non-plankton: {test_n_f0y0/test_n_y0:.3f} | test acc among plankton: {test_n_f1y1/test_n_y1:.3f}")

test acc among non-plankton: 0.973 | test acc among plankton: 0.506


In [44]:
# Run MAI, estimating confusion matrix
delta = 0.05

def invert_for_lb_0(r): return binom.cdf(n_f0y0,n_y0,r)-(1-delta/8)
def invert_for_lb_1(r): return binom.cdf(n_f1y1,n_y1,r)-(1-delta/8)
def invert_for_ub_0(r): return binom.cdf(n_f0y0,n_y0,r)-(delta/8)
def invert_for_ub_1(r): return binom.cdf(n_f1y1,n_y1,r)-(delta/8)

def invert_for_lb_f(r): return binom.cdf(N_f1,N,r)-(1-delta/8)
def invert_for_ub_f(r): return binom.cdf(N_f1,N,r)-(delta/8)

c0_lb = brentq(invert_for_lb_0,0,1)
c0_ub = brentq(invert_for_ub_0,0,1)

c1_lb = brentq(invert_for_lb_1,0,1)
c1_ub = brentq(invert_for_ub_1,0,1)

f1_lb = brentq(invert_for_lb_f,0,1)
f1_ub = brentq(invert_for_ub_f,0,1)

A_lb = np.array([[c0_lb, 1-c1_ub], [1-c0_lb, c1_ub]])
A_ub = np.array([[c0_ub, 1-c1_lb], [1-c0_ub, c1_lb]])

qyhat_lb = (np.linalg.inv(A_lb)@np.array([1-f1_lb, f1_lb]))
qyhat_ub = (np.linalg.inv(A_ub)@np.array([1-f1_ub, f1_ub]))

count_plankton_lb = binom.ppf(delta/8, N, qyhat_lb[1])
count_plankton_ub = binom.ppf(1-delta/8, N, qyhat_ub[1])

print(c0_lb,c0_ub,"\n")
print(c1_lb,c1_ub,"\n")
print(f1_lb,f1_ub,"\n")
print(A_lb,"\n\n",A_ub, "\n")
print(qyhat_lb, "\n", qyhat_ub)

0.9917684531839602 0.9924007643660543 

0.6586472505566562 0.6704499344785504 

0.11207529350406101 0.11544732958814871 

[[0.99176845 0.32955007]
 [0.00823155 0.67044993]] 

 [[0.99240076 0.34135275]
 [0.00759924 0.65864725]] 

[0.84318807 0.15681193] 
 [0.83434694 0.16565306]


In [45]:
print(f"The model-assisted confidence interval for the fraction of plankton observed in 2014 is [{qyhat_lb[1]:.4f},{qyhat_ub[1]:.4f}]; the true fraction was {test_labels.mean():.4f}")
print(f"The model-assisted confidence interval for the number of plankton observed in 2014 is [{int(count_plankton_lb)},{int(count_plankton_ub)}].")
print(f"The true number of plankton observed in 2014 was {test_labels.sum()}, which lies in the interval.")
print(f"The point estimate was {test_preds.sum()}, which does not lie in the interval.")

The model-assisted confidence interval for the fraction of plankton observed in 2014 is [0.1568,0.1657]; the true fraction was 0.1805
The model-assisted confidence interval for the number of plankton observed in 2014 is [34267,37087].
The true number of plankton observed in 2014 was 39945, which lies in the interval.
The point estimate was 25167, which does not lie in the interval.
